# EDA for the Medical Dataset

In [ ]:
!pip install rapidfuzz
!pip install datasets --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "

In [ ]:
# Standard library imports
import csv
import re
import string
from typing import List, Tuple, Optional, Dict

# Data analysis and NLP libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from difflib import SequenceMatcher
from rapidfuzz import fuzz
import spacy
from sentence_transformers import SentenceTransformer

# Scikit-learn
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from datasets import load_dataset

dataseries = load_dataset("rntc/mm-icd-notes")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/572 [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

train-00003-of-00006.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

train-00004-of-00006.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

train-00005-of-00006.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/122288 [00:00<?, ? examples/s]

In [ ]:
dataseries

DatasetDict({
    train: Dataset({
        features: ['subject_id', 'hadm_id', 'text', 'icd_code', 'cleaned_text', 'code_rare', 'code_commons', 'icd_descriptions'],
        num_rows: 122288
    })
})

In [ ]:
train_ds = dataseries['train']

In [ ]:
dataframe = pd.DataFrame(train_ds)

In [ ]:
dataframe.head()

,subject_id,hadm_id,text,icd_code,cleaned_text,code_rare,code_commons,icd_descriptions
0,10000084,23052089,\nName: ___ Unit No: __...,"[G3183, F0280, R441, R296, E785, Z8546]",Allergies: No Known Allergies / Adverse Drug R...,[],[E785],"[G3183: Dementia with Lewy bodies, F0280: Deme..."
1,10000117,22927623,\nName: ___ Unit No: ___\n...,"[R1310, R0989, K31819, K219, K449, F419, I341,...",Allergies: omeprazole Chief Complaint: dysphag...,[],"[K219, F419, Z87891]","[R1310: Dysphagia, unspecified, R0989: Other s..."
2,10000117,27988844,\nName: ___ Unit No: ___\n...,"[S72012A, W010XXA, Y93K1, Y92480, K219, E7800,...",Allergies: omeprazole / Iodine and Iodide Cont...,[],"[K219, Z87891, F419, Z7901]",[S72012A: Unspecified intracapsular fracture o...
3,10000980,20897796,\nName: ___ Unit No: ___\n \nAdmi...,"[I130, I5033, E872, N184, E1122, N2581, I2510,...",Allergies: No Known Allergies / Adverse Drug R...,[],"[I130, E872, E1122, I2510, E785, Z955, Z86718,...",[I130: Hypertensive heart and chronic kidney d...
4,10000980,25911675,\nName: ___ Unit No: ___\n \nAdmi...,"[D500, I5023, N184, E118, K2970, Z23, K259, K5...",Allergies: No Known Allergies / Adverse Drug R...,[],"[I2510, Z87891, I252, Z955, I129, Z794, Z8673,...",[D500: Iron deficiency anemia secondary to blo...


In [ ]:
req_cols = ['text', 'cleaned_text', 'icd_code', 'icd_descriptions']

df = dataframe[req_cols]

In [ ]:
df.head()

,text,cleaned_text,icd_code,icd_descriptions
0,\nName: ___ Unit No: __...,Allergies: No Known Allergies / Adverse Drug R...,"[G3183, F0280, R441, R296, E785, Z8546]","[G3183: Dementia with Lewy bodies, F0280: Deme..."
1,\nName: ___ Unit No: ___\n...,Allergies: omeprazole Chief Complaint: dysphag...,"[R1310, R0989, K31819, K219, K449, F419, I341,...","[R1310: Dysphagia, unspecified, R0989: Other s..."
2,\nName: ___ Unit No: ___\n...,Allergies: omeprazole / Iodine and Iodide Cont...,"[S72012A, W010XXA, Y93K1, Y92480, K219, E7800,...",[S72012A: Unspecified intracapsular fracture o...
3,\nName: ___ Unit No: ___\n \nAdmi...,Allergies: No Known Allergies / Adverse Drug R...,"[I130, I5033, E872, N184, E1122, N2581, I2510,...",[I130: Hypertensive heart and chronic kidney d...
4,\nName: ___ Unit No: ___\n \nAdmi...,Allergies: No Known Allergies / Adverse Drug R...,"[D500, I5023, N184, E118, K2970, Z23, K259, K5...",[D500: Iron deficiency anemia secondary to blo...


### Function to check whether the Chief Complaint is mentioned in the clinical note

In [ ]:
def contains_chief_complaint(text):
    pattern = r'(chief\s*complaint|cc|complaint)'
    return bool(re.search(pattern, text, re.IGNORECASE))

In [ ]:
df['Check'] = df['cleaned_text'].apply(contains_chief_complaint)

<ipython-input-12-055a333de657>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Check'] = df['cleaned_text'].apply(contains_chief_complaint)


In [ ]:
df['Check'].value_counts()

,count
Check,
True,122216
False,72


In [ ]:
# Removing all the rows for which there is no Chief Complaint

filtered_df = df[df['Check']]

In [ ]:
filtered_df['Check'].value_counts()

,count
Check,
True,122216


In [ ]:
filtered_df.drop('Check', axis=1, inplace=True)

<ipython-input-16-3a7991f11f40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop('Check', axis=1, inplace=True)


In [ ]:
filtered_df.head()

,text,cleaned_text,icd_code,icd_descriptions
0,\nName: ___ Unit No: __...,Allergies: No Known Allergies / Adverse Drug R...,"[G3183, F0280, R441, R296, E785, Z8546]","[G3183: Dementia with Lewy bodies, F0280: Deme..."
1,\nName: ___ Unit No: ___\n...,Allergies: omeprazole Chief Complaint: dysphag...,"[R1310, R0989, K31819, K219, K449, F419, I341,...","[R1310: Dysphagia, unspecified, R0989: Other s..."
2,\nName: ___ Unit No: ___\n...,Allergies: omeprazole / Iodine and Iodide Cont...,"[S72012A, W010XXA, Y93K1, Y92480, K219, E7800,...",[S72012A: Unspecified intracapsular fracture o...
3,\nName: ___ Unit No: ___\n \nAdmi...,Allergies: No Known Allergies / Adverse Drug R...,"[I130, I5033, E872, N184, E1122, N2581, I2510,...",[I130: Hypertensive heart and chronic kidney d...
4,\nName: ___ Unit No: ___\n \nAdmi...,Allergies: No Known Allergies / Adverse Drug R...,"[D500, I5023, N184, E118, K2970, Z23, K259, K5...",[D500: Iron deficiency anemia secondary to blo...


### Function for extracting Chief Complaint

In [ ]:
def extract_chief_complaint_section(text, start_keywords=None, end_keywords=None):
    if not isinstance(text, str):
        return ""

    if start_keywords is None:
        start_keywords = [
            r'chief\s*complaint',
            r'\[MASKED\]\s*complaint',
            r'\b___ complaint\b',
            r'\b___\b',
        ]
    if end_keywords is None:
        end_keywords = [
            r'\bmajor\b',
            r'\bdiagnosis\b',
            r'\bassessment\b',
            r'\[\s*MASKED\s*\]',
            r'\.'
        ]

    # Build regex patterns that ignore trailing colons or dashes
    start_pattern = r'(' + '|'.join(start_keywords) + r')\s*[:\-]?\s*'
    end_pattern = r'(' + '|'.join(end_keywords) + r')'

    pattern = re.compile(rf'{start_pattern}(.*?){end_pattern}', flags=re.IGNORECASE | re.DOTALL)
    match = pattern.search(text)

    if match:
        return match.group(2).strip()  # Extracted text between start and end
    return ""

In [ ]:
filtered_df['chief_complaint'] = filtered_df['cleaned_text'].apply(extract_chief_complaint_section)

<ipython-input-19-c26624879618>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['chief_complaint'] = filtered_df['cleaned_text'].apply(extract_chief_complaint_section)


In [ ]:
filtered_df['chief_complaint'] = filtered_df['chief_complaint'].replace(['', None], np.nan)

<ipython-input-20-49d0b03d980e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['chief_complaint'] = filtered_df['chief_complaint'].replace(['', None], np.nan)


In [ ]:
replaced_indices = filtered_df[filtered_df['chief_complaint'].isna()].index.tolist()

In [ ]:
filtered_df['chief_complaint'].isna().sum()

np.int64(4669)

In [ ]:
filtered_df.dropna(inplace=True)

<ipython-input-23-28e735433b1f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.dropna(inplace=True)


In [ ]:
filtered_df['chief_complaint'].isna().sum()

np.int64(0)

In [ ]:
filtered_df.shape

(117547, 5)

In [ ]:
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
filtered_df.head()

,text,cleaned_text,icd_code,icd_descriptions,chief_complaint
0,\nName: ___ Unit No: __...,Allergies: No Known Allergies / Adverse Drug R...,"[G3183, F0280, R441, R296, E785, Z8546]","[G3183: Dementia with Lewy bodies, F0280: Deme...",Visual hallucinations
1,\nName: ___ Unit No: ___\n...,Allergies: omeprazole Chief Complaint: dysphag...,"[R1310, R0989, K31819, K219, K449, F419, I341,...","[R1310: Dysphagia, unspecified, R0989: Other s...",dysphagia
2,\nName: ___ Unit No: ___\n...,Allergies: omeprazole / Iodine and Iodide Cont...,"[S72012A, W010XXA, Y93K1, Y92480, K219, E7800,...",[S72012A: Unspecified intracapsular fracture o...,Left hip pain
3,\nName: ___ Unit No: ___\n \nAdmi...,Allergies: No Known Allergies / Adverse Drug R...,"[I130, I5033, E872, N184, E1122, N2581, I2510,...",[I130: Hypertensive heart and chronic kidney d...,shortness of breath
4,\nName: ___ Unit No: ___\n \nAdmi...,Allergies: No Known Allergies / Adverse Drug R...,"[D500, I5023, N184, E118, K2970, Z23, K259, K5...",[D500: Iron deficiency anemia secondary to blo...,"fatigue, anemia"


In [ ]:
# An idea on Maximum and Average Length of the Chief Complaints

length = []

for i in range(len(filtered_df)):
    length.append(len(filtered_df['chief_complaint'].iloc[i]))

print('avg length:', sum(length)/len(length))
print('max length:', max(length))
print('max length index:', length.index(max(length)))
print('min length:', min(length))

avg length: 21.69069393519188
max length: 372
max length index: 38938
min length: 1


In [ ]:
filtered_df['cleaned_text'].iloc[38938]

"Allergies: tree nuts Chief Complaint: Single intrauterine pregnancy at 41 week 2 days, gestational hypertension, arrest of dilation, live born male, spontaneous separation of the serosal surface and part of the myometrium of the lower anterior uterine segment extending laterally into the anterior portion of the right broad ligament and inferiorly into the posterior wall of the bladder, postpartum hemorrhage. Major Surgical or Invasive Procedure: primary transverse cesarean section cystotomy of the bladder and repair by urology team Physical Exam: Physical Exam on Discharge: VS: Afebrile, Vital signs stable Neuro/Psych: no acute distress, Oriented x3, Affect Normal Heart: regular rate and rhythm Lungs: clear to auscultation bilaterally Abdomen: soft, appropriately tender, fundus firm, incision clean/dry/intact Pelvis: minimal bleeding Extremities: warm and well perfused, no calf tenderness, no edema Pertinent Results: Labs on Admission: [MASKED] 04:55AM BLOOD WBC-15.9* RBC-4.00 Hgb-12.

In [ ]:
filtered_df['chief_complaint'].iloc[38938]

'Single intrauterine pregnancy at 41 week 2 days, gestational hypertension, arrest of dilation, live born male, spontaneous separation of the serosal surface and part of the myometrium of the lower anterior uterine segment extending laterally into the anterior portion of the right broad ligament and inferiorly into the posterior wall of the bladder, postpartum hemorrhage'

### Preprocessing the Chief Complaint Feature

In [ ]:
def preprocess_chief_complaint(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""

    text = text.lower() # Convert to lowercase
    text = re.sub(r"[^a-z0-9\s]", " ", text) # Remove unwanted characters and symbols
    text = re.sub(r"\s+", " ", text).strip() # Remove extra whitespace

    return text

In [ ]:
filtered_df['chief_complaint'] = filtered_df['chief_complaint'].apply(preprocess_chief_complaint)

<ipython-input-32-5f9e8d4f2886>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['chief_complaint'] = filtered_df['chief_complaint'].apply(preprocess_chief_complaint)


In [ ]:
filtered_df.columns

Index(['text', 'cleaned_text', 'icd_code', 'icd_descriptions',
       'chief_complaint'],
      dtype='object')

### Chief Complaint Extraction

In [ ]:
chief_complaints_file = filtered_df[['chief_complaint', 'icd_code']]

In [ ]:
type(chief_complaints_file)

pandas.core.frame.DataFrame

In [ ]:
chief_complaints_file.head()

,chief_complaint,icd_code
0,visual hallucinations,"[G3183, F0280, R441, R296, E785, Z8546]"
1,dysphagia,"[R1310, R0989, K31819, K219, K449, F419, I341,..."
2,left hip pain,"[S72012A, W010XXA, Y93K1, Y92480, K219, E7800,..."
3,shortness of breath,"[I130, I5033, E872, N184, E1122, N2581, I2510,..."
4,fatigue anemia,"[D500, I5023, N184, E118, K2970, Z23, K259, K5..."


In [ ]:
chief_complaints_file.to_csv('chief_complaints.csv', index=False)

## Data Extraction for Finetuning

In [ ]:
finetune_df = filtered_df[['cleaned_text', 'icd_code']]

In [ ]:
finetune_df.head()

,cleaned_text,icd_code
0,Allergies: No Known Allergies / Adverse Drug R...,"[G3183, F0280, R441, R296, E785, Z8546]"
1,Allergies: omeprazole Chief Complaint: dysphag...,"[R1310, R0989, K31819, K219, K449, F419, I341,..."
2,Allergies: omeprazole / Iodine and Iodide Cont...,"[S72012A, W010XXA, Y93K1, Y92480, K219, E7800,..."
3,Allergies: No Known Allergies / Adverse Drug R...,"[I130, I5033, E872, N184, E1122, N2581, I2510,..."
4,Allergies: No Known Allergies / Adverse Drug R...,"[D500, I5023, N184, E118, K2970, Z23, K259, K5..."


In [ ]:
finetune_df_1k = finetune_df.iloc[:1000]
finetune_df_1k.to_csv('finetune_df_1k.csv', index=False)

In [ ]:
finetune_df_30k = finetune_df.iloc[:30000]

In [ ]:
finetune_df_30k.to_csv('finetune_df_30k.csv', index=False)

In [ ]:
finetune_df_10k = finetune_df.iloc[:10000]

In [ ]:
finetune_df_10k.to_csv('finetune_df_10k.csv', index=False)